In [1]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


## 특성 스케일 바꾸기

### Minmaxscaler : 최소 최대 스케일링

In [4]:
import numpy as np
from sklearn import preprocessing

In [6]:
#스케일러 객체를 만든다

In [7]:
feature = np.array([[-500.5],[-100.1],[0],[100.1],[900.9]])
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))

In [12]:
scaled_feature = minmax_scale.fit_transform(feature) #fit 과 transform 을 한꺼번에 해서 훈련/ 조정을 한 번에

In [13]:
#0~1사이의 결과 반환

In [14]:
scaled_feature

array([[0.        ],
       [0.28571429],
       [0.35714286],
       [0.42857143],
       [1.        ]])

### Robustscaler : 이상치가 많을 때 스케일링. 데이터에서 중간값을 빼고 IQR(75%의값 - 25%의값)로 나눈다.

In [17]:
x= np.array([[-1000.1],[-200.2],[500.5],[600.6],[9000.9]])

In [18]:
robust_scaler = preprocessing.RobustScaler()
robust_scaler.fit_transform(x)

array([[-1.87387612],
       [-0.875     ],
       [ 0.        ],
       [ 0.125     ],
       [10.61488511]])

### 정규화하기 : 평균이 0 표준편차가 1로 맞춘다.


In [20]:
from sklearn.preprocessing import Normalizer

In [21]:
features = np.array([[0.5, 0.5],
                    [1.1,3.4],
                    [1.5,20.2],
                    [1.63,34.4],
                    [10.9, 3.3]])

In [22]:
features.shape

(5, 2)

In [25]:
# 유클리드 노름이 기본값
normalizer = Normalizer(norm ="l2")

In [26]:
normalizer.transform(features)

array([[0.70710678, 0.70710678],
       [0.30782029, 0.95144452],
       [0.07405353, 0.99725427],
       [0.04733062, 0.99887928],
       [0.95709822, 0.28976368]])

## 다항 특성과 교차항 특성 생성하기

###  - polynomial 특성과 interaction 특성을 만들고 싶습니다.

###### 다항 특성을 쓰는 경우 : 특성과 타깃 사이 비선형 관계가 있을 때

In [27]:
from sklearn.preprocessing import PolynomialFeatures

In [28]:
features = np.array([[2,3],
                    [2,3],
                    [2,3]])

In [29]:
polynomial_interaction = PolynomialFeatures(degree=2, include_bias = False)

In [30]:
polynomial_interaction.fit_transform(features)

array([[2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.],
       [2., 3., 4., 6., 9.]])

In [35]:
interaction = PolynomialFeatures (degree=3, interaction_only=True, include_bias =True) 
#include_bias 는 1이 포함, degree는 3차 , interaction_only는 교차항만 , degree 는 최대 차수를 볼래
interaction.fit_transform(features)

array([[1., 2., 3., 6.],
       [1., 2., 3., 6.],
       [1., 2., 3., 6.]])

## 특성 변환하기

### 하나 이상의 특성에 사용자 정의 변환을 적용하고 싶습니다.

In [36]:
from sklearn.preprocessing import FunctionTransformer

In [39]:
features = np.array([[2,3],
                   [2,3],
                   [2,3]])

In [40]:
features.shape

(3, 2)

In [41]:
def add_ten(x):
    return x +10

In [42]:
ten_transformer = FunctionTransformer(add_ten)

In [43]:
ten_transformer.transform(features)

array([[12, 13],
       [12, 13],
       [12, 13]])

#### 열마다 다른 사용자 정의 변환을 사용하고 싶다.

In [46]:
from sklearn.compose import ColumnTransformer
import pandas as pd

In [51]:
#각열별로 사용자 정의 변환을 하려면 데이터프레임에 열이름을 지정해줘야한다.

df= pd.DataFrame(features, columns=['feature_1','feature_2'])


In [48]:
def add_hundred(x):
    return x +100

In [49]:
ct= ColumnTransformer([("add_ten",FunctionTransformer(add_ten, validate=True),['feature_1']),
                      ("add_hundred", FunctionTransformer(add_hundred, validate=True),['feature_2'])])

In [50]:
ct.fit_transform(df)

array([[ 12, 103],
       [ 12, 103],
       [ 12, 103]])

## 이상치 감지하기

In [52]:
# 타원 안의 샘플을 레이블 1로 구분하고, 타원 밖의 샘플을 레이블 -1 로 구분한다.

In [64]:
from sklearn.covariance import EllipticEnvelope # 타원 밖의 이상치 탐지하기
from sklearn.datasets import make_blobs # 모의 데이터 만들기

In [65]:
# features는 입력 변수 입력 변수 features(열이 2개), 얼룩의 중심은 하나
features, _ = make_blobs(n_samples=10, n_features =2, centers=1, random_state=1) 


In [66]:
#이상치 만들기

features[0,0]=10000
features[0,1]=10000

In [67]:
#contamination=0.1 10%의 이상치 비율
outlier_detector = EllipticEnvelope(contamination=.1)
outlier_detector.fit(features)
outlier_detector.predict(features)

array([-1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [70]:
feature = features[:,0]

In [71]:
# 이상치의 인덱스를 반환하는 함수

def indicies_of_outliers(x):
    q1, q3 = np.percentile(x,[25,75])
    iqr= q3- q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    return np.where((x> upper_bound)| (x<lower_bound))


In [73]:
print(indicies_of_outliers(feature))

(array([0], dtype=int64),)


## 이상치 다루기 

In [74]:
import pandas as pd

In [75]:
houses = pd.DataFrame()

# 네번째 데이터를 이상치로 만든다

houses['price']=[534433,392333,293222,4322032]
houses['bathrooms']=[2,3.5,2,116]
houses['square_feet']=[1500,2500,1500,48000]

1. 이상치 삭제

In [77]:
houses[houses['bathrooms']<20]

,price,bathrooms,square_feet
0,534433,2.0,1500
1,392333,3.5,2500
2,293222,2.0,1500


2. 이상치 열 추가

In [80]:
houses['outlier']= np.where(houses['bathrooms']<20,0,1)
houses

,price,bathrooms,square_feet,outlier
0,534433,2.0,1500,0
1,392333,3.5,2500,0
2,293222,2.0,1500,0
3,4322032,116.0,48000,1


3. 이상치 영향이 줄어들도록 특성을 변환한다. (log transformation)

In [83]:
houses['log_of_square_feet']=[np.log(x) for x in houses['square_feet']]
houses

,price,bathrooms,square_feet,outlier,log_of_square_feet
0,534433,2.0,1500,0,7.313220
1,392333,3.5,2500,0,7.824046
2,293222,2.0,1500,0,7.313220
3,4322032,116.0,48000,1,10.778956


## 특성 이산화하기 (Binarizer, digitize 수치 특성을 범주형처럼 다루기)

chatgpt가 말하는 수치형 -> 범주형 변수로 변환의 이점 
1. 비선형 관계 처리
2. 노이즈 및 중복성 감소 ( 18세 이상, 50세 미만 등으로 범주를 나누면 데이터를 단순화할 수 있다.)

In [88]:
from sklearn.preprocessing import Binarizer 

In [89]:
age= np.array([[6],[12],[20],[36],[65]])

In [90]:
binarizer =Binarizer(18)
binarizer.fit_transform(age)

C:\Users\82109\anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass threshold=18 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


array([[0],
       [0],
       [1],
       [1],
       [1]])

In [91]:
np.digitize(age, bins=[20,30,64])

array([[0],
       [0],
       [1],
       [2],
       [3]], dtype=int64)

## 구간으로 나누는 또 다른 방법 KBinsDiscretizer

In [92]:
from sklearn.preprocessing import KBinsDiscretizer

In [94]:
# 네 개의 구간으로 나눈다.
kb= KBinsDiscretizer(4, encode='ordinal', strategy='quantile')
kb.fit_transform(age)

array([[0.],
       [1.],
       [2.],
       [3.],
       [3.]])

In [95]:
kb= KBinsDiscretizer(4, encode='onehot-dense', strategy='quantile')
kb.fit_transform(age)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [96]:
kb= KBinsDiscretizer(4, encode='onehot-dense', strategy='uniform')
kb.fit_transform(age)

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 군집으로 샘플을 그룹으로 묶기(139페이지)